#### Michele Meazzini - 635889
# Songs Genre Classifier
This project aims to speed up the process of classifying songs by genre using different types of classification models. In this code, the input audio files are split into 3 seconds tracks, which are then analyzed and classified by their genre choosing from 10 genres: blues, classical, country, disco, hiphop, jazz, metal, pop, reggae and rock. Different tecniques of data visualization, preprocessing and classification are involved to achieve the best performances possible in terms of accuracy, while trying to reduce the amount of necessary data to make the dataset lighter and speed up the classification process.

## Table of Contents
1. [Imports](#Imports)
2. [Feature Extraction](#Feature-Extraction)
3. [Data Exploration](#Data-Exploration)
4. [Data Preprocessing](#Data-Preprocessing)
5. [Classification](#Classification)


# Imports

## Feature Extraction

## Data Exploration

### Boxplots (tempo)

## Data Preprocessing

### Feature Selection based on Correlation

### Feature Normalization 

## Classification

### Confusion Matrix

### Recursive Feature Elimination

## Model Selection